In [96]:
#Import necessary libraries
import os
from dotenv import load_dotenv
import json
import requests
from typing import Dict, List

load_dotenv(dotenv_path=".env")

#Constants
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

#Main API routes
SPOTIFY_API_URL_PREFIX = "https://api.spotify.com/v1"

In [97]:
def get_token(client_id:str=SPOTIFY_CLIENT_ID, client_secret:str=SPOTIFY_CLIENT_SECRET):
    """
    Retrieves a Spotify access token.

    Params:
        client_id (str): The Spotify Client ID.
        client_secret (str): The Spotify Client Secret.

    Returns:
        response_json (Dict(str, str, int)): A dictionary containing:
            - access_token (str): The Spotify access token.
            - token_type (str): The type of authorization token (e.g., "Bearer").
            - expires_in (int): The token's expiration time in seconds (typically 3600).
    """
    URL_TOKEN = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type":"application/x-www-form-urlencoded"
    }
    payload ={
        "grant_type":"client_credentials",
        "client_id":client_id,
        "client_secret":client_secret
    }
    try:
        response = requests.post(url=URL_TOKEN, headers=headers, data=payload)
        response_json = json.loads(response.content)
        return response_json
    except Exception as err:
        print(err)
        return {}

def get_auth_header(token: Dict[str,any]) -> Dict[str, str]:
    """
    Retrieves Spotify authorization header
    
    Params:
        token (Dict[str,any]) : token dictionary from get_token()
        
    Returns:
        authorization_header (Dict(str,str)): Bearer authorization header

    """
    authorization_header = {"Authorization": f"Bearer {token['access_token']}"}
    return authorization_header   

## Extract

In [124]:

def get_tracks_by_genre(genre:str, token:Dict[str,any],offset:int=0,limit:int=20):
    """
    Fetch tracks by genre from Spotify
    
    Params:
        genre (str): The genre to search for
        token (Dict[str,any]): Spotify authentication token
        offset (int, optiional): Pagination offset, Default to 0
        limit (int, optional): Number of tracks to fetch per request. Defaults to 20
    
    Returns:
        tracks (Dict[str,any]): A response detail from Spotify API
    
    """
    ENDPOINT = f"{SPOTIFY_API_URL_PREFIX}/search?q=genre:{genre}&type=track&offset={offset}&limit={limit}"
    headers = get_auth_header(token=token)
    response = requests.get(url=ENDPOINT, headers=headers)
    tracks = response.json()['tracks']
    return tracks

def get_track_details_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves a list of track details 
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        track_details (List[Dict[str,any]]): A list of dictionaries of new tracks details, containing:
            - id (str): Spotify ID for the track
            - name (str): Name of this track
            - url (str): Spotify URL for the object
            - duration_ms (int): The track length in milliseconds
            - artist_ids (List[str]): List of Spotify ID for the artists
            - album_id (str): Spotify ID for the album
            - release_date (str): The date the album was first released
            - is_single (Boolean):Check if this is a single or part of the album
            - explicit (Boolean): Whether or not the track has explicit lyrics
    """
    track_details = []
    for item in tracks['items']:
        track_artist_ids =[artist['id'] for artist in item['artists']]
        track_detail = {
            "id":item["id"],
            "name":item['name'],
            "url":item['external_urls']['spotify'],
            "duration_ms":item['duration_ms'],
            'artist_ids':track_artist_ids,
            "album_id":item['album']['id'],
            'release_date':item['album']['release_date'],
            "is_single":True if item['album']['album_type']=='single' else False,
            'explicit': item['explicit']
        }
        track_details.append(track_detail)
        
    return track_details
    
def get_album_details_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves a list of album details from tracks
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        album_details (List[Dict[str,any]]): A list of dictionaries of new album details, containing:
            - id (str): Spotify ID for the album
            - album_name (str): The name of the album. In case of an album takedown, the value may be an empty string
            - release_date (str): The date the album was first released
            - artist_ids (List[str]): List of Spotify ID for the artists
            - total_tracks (int): number of tracks in the album
            - image_640_url (str): URL for the album cover size 640x640
            _ image_300_url (str): URL for the album cover size 300x300
            _ image_64_url (str): URL for the album cover size 64x64
    """
    album_details = []
    for item in tracks['items']:
        album = item['album']
        album_artist_ids = [artist['id'] for artist in album['artists']]
        album_detail= {
            "id":album['id'],
            "name":album['name'],
            "release_date":album['release_date'],
            'artist_ids':album_artist_ids,
            'album_url':album['external_urls']['spotify'],
            'total_tracks':album['total_tracks'],
            "type":album['type'],
            "image_640_url":album['images'][0]["url"] if album['images'] else "",
            "image_300_url":album['images'][1]["url"] if album['images'] else "",
            'image_64_url':album['images'][2]['url'] if album['images'] else ""
        }
        album_details.append(album_detail)
        
    return album_details

def get_artist_track_dict_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves artist who procduces the track
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        artist_track_dict (Dict[str,List[str]]): A dictionary containing artist_id own a list of track
    """
    artist_track_dict:Dict[str,List[str]] = {}
    for item in tracks['items']:
        track_id = item['id']
        for artist in item['artists']:
            artist_id = artist['id']
            if artist_track_dict.get(artist_id) is None:
                artist_track_dict[artist_id] = []
            artist_track_dict[artist_id].append(track_id)
    return artist_track_dict
        
def get_artist_album_dict_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves artist who procduces the album
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        artist_album_dict (Dict[str,List[str]]): A dictionary containing artist_id own a list of album
    """
    
    artist_album_dict:Dict[str,List[str]] = {}
    for item in tracks['items']:
        album = item['album']
        album_id = album['id']
        for artist in item['artists']:
            artist_id = artist['id']
            if artist_album_dict.get(artist_id) is None:
                artist_album_dict[artist_id] = []
            artist_album_dict[artist_id].append(album_id)
    return artist_album_dict
        
def get_artist_detail(artist_id:str, token:Dict[str,any]):
    '''
    Retrieve artist information from Spotify
    
    Params:
        artist_id (str): Spotify Id of the artist
    
    Returns:
        artist_detail (Dict[str,str]): A dictionary of artist detail, containing:
            - artist_id (str): Spotify ID for the artist
            - name (str): The name of the artist
            - url (str): Spotify URL for the object
            - genres (List[str]): A list of the genres thr artist is associated with. If not yet classified, the array is empty
            - image_640_url (str): URL for the artist cover size 640x640
            - image_320_url (str): URL for the artist cover size 320x320
            - image_160_url (str): URL for the artist cover size 160x160
        
    '''
    ENDPOINT = f"{SPOTIFY_API_URL_PREFIX}/artists/{artist_id}"
    headers = get_auth_header(token=token)
    response = requests.get(url=ENDPOINT, headers=headers)
    artist_doc = response.json()
    artist_detail = {
        "artist_id": artist_doc['id'],
        "name": artist_doc['name'],
        "url":artist_doc['external_urls']['spotify'],
        'genres':artist_doc['genres'],
        'image_640_url':artist_doc['images'][0]['url'] if artist_doc['images'] else "",
        'image_320_url':artist_doc['images'][1]['url'] if artist_doc['images'] else "",
        'image_160_url':artist_doc['images'][2]['url'] if artist_doc['images'] else ""
    }
    return artist_detail
    
def get_genre_list_from_artist(artist_details:List[Dict[str,any]]):
    '''
    Retrieve list of genres of artists
    
    Params:
        artist_details (List[Dict[str,any]]): details of artist 
    
    Returns:
        genre_list (List): list of unique genres
    '''
    genre_list = []
    for artist_detail in artist_details:
        genre_list.extend(artist_detail['genres'])
    return list(set(genre_list))

def get_artist_ids_from_tracks(tracks):
    artists_from_album = []
    artists_from_artists = []
    for item in tracks['items']:
        for artist in item['album']['artists']:
           artists_from_album.append(artist['id'])
        for artist in item['artists']:
            artists_from_artists.append(artist['id'])
    artist_ids = list(set(artists_from_album + artists_from_artists))
    return artist_ids
def get_artist_details_from_ids(artist_ids:List[str]):
    token = get_token()
    artist_details = [get_artist_detail(artist_id, token=token) for artist_id in artist_ids]
    return artist_details
def get_artist_genre_from_artist_details(artist_details):
    artist_genres_dict = {artist_detail['artist_id']:artist_detail['genres'] for artist_detail in artist_details}
    return artist_genres_dict

def get_track_album_from_tracks(tracks):
    track_album = {}
    for item in tracks['items']:
        track_id = item['id']
        album_id = item['album']['id']
        if track_album.get(track_id) is None:
            track_album[track_id] = album_id
    return track_album
    

## Transform

In [263]:
import pandas as pd

def create_track_df(tracks):
    track_details = get_track_details_from_tracks(tracks)
    
    track_df = pd.DataFrame(track_details).drop(columns=["artist_ids","album_id"]).drop_duplicates()
    track_df['release_date'] = pd.to_datetime(track_df['release_date'], format='mixed')
    return track_df

def create_album_df(tracks):
    album_details = get_album_details_from_tracks(tracks)
    album_df = pd.DataFrame(album_details).drop(columns=['artist_ids']).drop_duplicates()
    album_df['release_date']=pd.to_datetime(album_df['release_date'], format='mixed')
    
    return album_df


def create_artist_track_df(tracks):
    artist_tracks = get_artist_track_dict_from_tracks(tracks)
    artist_tracks_df = pd.DataFrame({"artist_id": list(artist_tracks.keys()), "track_id": artist_tracks.values()}).explode("track_id")
    return artist_tracks_df

def create_artist_album_df(tracks):
    artist_albums = get_artist_album_dict_from_tracks(tracks)
    artist_albums_df = pd.DataFrame({"artist_id": list(artist_albums.keys()), "album_id": artist_albums.values()}).explode("album_id")
    return artist_albums_df


"""Instead of creating from tracks, these functions create from artist details. Because if create from tracks, the artist information is not enough so the better approach is to get artist ids then find the details. 
    The process of finding details takes long time so it is not sufficient to do it everytime the function is called
    ```python
    artist_ids = get_artist_ids_from_tracks(tracks)
    artist_details = get_artist_details_from_ids(artist_ids)
    ```
    """
def create_artist_df(artist_details):
    artist_df = pd.DataFrame(artist_details).drop(columns='genres').drop_duplicates()
    return artist_df

def create_artist_genre_df(artist_details):
    artist_genres = get_artist_genre_from_artist_details(artist_details)
    artist_genres_df = pd.DataFrame({"artist_id":artist_genres.keys(),"genre":artist_genres.values()}).explode('genre')
    return artist_genres_df

def create_genre_df(artist_details):
    '''
    DEPRECATED
    '''
    genre_list = get_genre_list_from_artist(artist_details)
    genre_df = pd.DataFrame(genre_list).drop_duplicates()
    return genre_df
    
def create_track_album_df(track_in_album):
    track_album_df = pd.DataFrame(list(track_in_album.items()), columns=['track_id','album_id'])
    return track_album_df


## Load

In [249]:
from sqlalchemy import create_engine
import sqlalchemy
load_dotenv(dotenv_path='.env')
DB_URL = os.getenv('DB_URL')
engine = create_engine(DB_URL)

In [245]:
del DB_URL
del os.environ['DB_URL']

In [250]:
#Extract data
token = get_token()
offset, limit = 0,50
tracks = get_tracks_by_genre(genre='dubstep', token=token, offset=offset, limit=limit)
track_df = create_track_df(tracks)
album_df = create_album_df(tracks)
artist_albums_df = create_artist_album_df(tracks)
artist_tracks_df = create_artist_track_df(tracks)
artist_ids = get_artist_ids_from_tracks(tracks)
artist_details = get_artist_details_from_ids(artist_ids)
artist_df = create_artist_df(artist_details)
artist_genres_df = create_artist_genre_df(artist_details)

track_album = get_track_album_from_tracks(tracks)
track_album_df = create_track_album_df(track_album)


In [313]:
def get_existing_values_df(engine:sqlalchemy.engine.base.Engine, table_name:str):
    if table_name not in ["track", "artist", "album"]:
        raise ValueError(f"Table name {table_name} is not supported")
    with engine.begin() as conn:
        try:
            existing_values = pd.read_sql(con=conn, sql=f'SELECT id FROM {table_name}')  
            return existing_values
        except Exception as e:
            print(e)
            return None
def filter_duplicate(df1:pd.DataFrame, df2:pd.DataFrame):
    return df1.merge(df2, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

def load_to_db(df: pd.DataFrame, engine: sqlalchemy.engine.base.Engine, table_name: str):
    existing_values_df = get_existing_values_df(engine=engine, table_name=table_name)
    #Filter out duplicate id and keep the unique one
    filtered_df = filter_duplicate(df, existing_values_df)
    try:
        with engine.connect() as conn:
            filtered_df.to_sql(name=table_name, con=conn, if_exists='append', index=False)
    except Exception as err:
        print(err)



In [26]:
import utils



Extract
